<a href="https://colab.research.google.com/github/mrbenbot/wimblepong/blob/main/Wimblepong_Reinforcement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Setup Google Colab Environment
%pip install gym
%pip install stable-baselines3[extra]
%pip install imageio pillow
%pip install tensorflowjs




: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

: 

In [ ]:
# Import necessary libraries
from gymnasium import spaces
import gymnasium as gym

import stable_baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from IPython import display

import os
import imageio
import glob
import math

from IPython.display import display, Image, HTML

import tensorflow as tf
from tensorflow.keras import layers
import torch as th
import numpy as np


# Check versions
print("gym version:", gym.__version__)
print("stable-baselines3 version:", stable_baselines3.__version__)


: 

In [ ]:
COURT_HEIGHT = 800
COURT_WIDTH = 1200
PADDLE_HEIGHT = 90
PADDLE_WIDTH = 15
BALL_RADIUS = 12
INITIAL_BALL_SPEED = 10
PADDLE_SPEED_DIVISOR = 15  # Example value, adjust as needed
PADDLE_CONTACT_SPEED_BOOST_DIVISOR = 4  # Example value, adjust as needed
SPEED_INCREMENT = 0.6  # Example value, adjust as needed
SERVING_HEIGHT_MULTIPLIER = 2  # Example value, adjust as needed
PLAYER_COLOURS = {'Player1': 'blue', 'Player2': 'red'}
MAX_COMPUTER_PADDLE_SPEED = 10

: 

In [ ]:
rewards_map = {
    "hit_paddle": lambda _: 50,
    "score_point": lambda _: 100,
    "conceed_point": lambda ball, paddle, rally_length: -abs(ball['y'] - paddle['y']) / max(rally_length, 1),
    "serve": lambda ball_speed: ball_speed / 10,
    "paddle_movement": lambda dy: abs(dy) / 15,
    "ball_distance": lambda ball, paddle: -abs(ball['y'] - (paddle['y'] + PADDLE_HEIGHT))
}

: 

In [ ]:



class Player:
    Player1 = 'Player1'
    Player2 = 'Player2'

class PlayerPositions:
    Initial = 'Initial'
    Reversed = 'Reversed'

class GameEventType:
    ResetBall = 'ResetBall'
    Serve = 'Serve'
    WallContact = 'WallContact'
    HitPaddle = 'HitPaddle'
    ScorePointLeft = 'ScorePointLeft'
    ScorePointRight = 'ScorePointRight'

def get_bounce_angle(paddle_y, paddle_height, ball_y):
    relative_intersect_y = (paddle_y + (paddle_height / 2)) - ball_y
    normalized_relative_intersect_y = relative_intersect_y / (paddle_height / 2)
    return normalized_relative_intersect_y * (math.pi / 4)

class CustomPongEnv(gym.Env):
    def __init__(self):
        super(CustomPongEnv, self).__init__()
        self.action_space = spaces.Box(low=np.array([0, -60]), high=np.array([1, 60]), dtype=np.float32)
        # i have updated this. Please provide me with just the observation space declaration please

            # float(state['ball']['x']),
            # float(state['ball']['y']),
            # float(state['ball']['dx']),
            # float(state['ball']['dy']),
            # float(paddle['x']),
            # float(paddle['y']),
            # float(int(state['ball']['serve_mode'])),
            # float(is_server),

        self.observation_space = spaces.Box(
            low=np.array([0, 0, -np.inf, -np.inf, 0, 0, 0, 0], dtype=np.float32),
            high=np.array([COURT_WIDTH, COURT_HEIGHT, np.inf, np.inf, COURT_WIDTH, COURT_HEIGHT, 1, 1], dtype=np.float32)
        )
        # Define the possible starting states
        self.starting_states = [
           {'server': Player.Player2, 'positions_reversed': False, 'opponent': Player.Player1, 'player': Player.Player2},
           {'server': Player.Player2, 'positions_reversed': True, 'opponent': Player.Player1, 'player': Player.Player2},
           {'server': Player.Player1, 'positions_reversed': False, 'opponent': Player.Player1, 'player': Player.Player2},
           {'server': Player.Player1, 'positions_reversed': True, 'opponent': Player.Player1, 'player': Player.Player2},
        ]

        self.serve_delay = 50
        self.serve_delay_counter = 0
        self.direction = 15
        self.is_done = False
        self.frame_count = 0

        self.last_event = None

        # Initialize the game state
        self.reset(seed=0)

    def seed(self, seed=None):
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        return [seed]

    def reset(self, seed=None):
        super().reset(seed=seed)
        if seed is not None:
            self.seed(seed)
        # Select a random starting state
        starting_state = np.random.choice(self.starting_states)

        # Set the server, positions, and computer player based on the starting state
        server = starting_state['server']
        positions_reversed = starting_state['positions_reversed']
        computer = starting_state['opponent']
        player = starting_state['player']

        # Initialize the game state
        self.state = {
            'server': server,
            'positions_reversed': positions_reversed,
            'opponent': computer,
            'player': player,
            Player.Player1: {'x': 0, 'y': COURT_HEIGHT // 2 - PADDLE_HEIGHT // 2, 'dy': 0, 'width': PADDLE_WIDTH, 'height': PADDLE_HEIGHT, 'colour': 'blue'},
            Player.Player2: {'x': COURT_WIDTH - PADDLE_WIDTH, 'y': COURT_HEIGHT // 2 - PADDLE_HEIGHT // 2, 'dy': 0, 'width': PADDLE_WIDTH, 'height': PADDLE_HEIGHT, 'colour': 'red'},
            'ball': {'x': COURT_WIDTH // 2, 'y': COURT_HEIGHT // 2, 'dx': INITIAL_BALL_SPEED, 'dy': INITIAL_BALL_SPEED, 'radius': BALL_RADIUS, 'speed': INITIAL_BALL_SPEED, 'serve_mode': True, 'score_mode': False, 'score_mode_timeout': 0},
            'stats': {'rally_length': 0, 'serve_speed': INITIAL_BALL_SPEED, 'server': server}
        }
        self.is_done = False

        # Apply the meta game state adjustments
        self.apply_meta_game_state()
        self.serve_delay_counter = 0
        self.direction = 30 * np.random.rand()
        self.serve_delay = 100 * np.random.rand()
        self.direction = self.direction if np.random.rand() > 0.5 else -self.direction

        # Initialize frame count and ensure frames directory exists
        # self.frame_count = 0
        if not os.path.exists('frames'):
            os.makedirs('frames')
        return self._get_obs(), {}

    def apply_meta_game_state(self):
        game_state = self.state
        serving_player = game_state['server']
        positions_reversed = game_state['positions_reversed']

        # Set paddle heights
        if serving_player == Player.Player1:
            self.state[Player.Player1] = PADDLE_HEIGHT * SERVING_HEIGHT_MULTIPLIER
            self.state[Player.Player2] = PADDLE_HEIGHT
        else:
            self.state[Player.Player1] = PADDLE_HEIGHT
            self.state[Player.Player2] = PADDLE_HEIGHT * SERVING_HEIGHT_MULTIPLIER

        # Set paddle positions
        if positions_reversed:
            self.state[Player.Player1]['x'] = COURT_WIDTH - PADDLE_WIDTH
            self.state[Player.Player2]['x'] = 0
        else:
            self.state[Player.Player1]['x'] = 0
            self.state[Player.Player2]['x'] = COURT_WIDTH - PADDLE_WIDTH

        ball = self.state['ball']
        server_is_left = (serving_player == Player.Player1 and not positions_reversed) or (serving_player == Player.Player2 and positions_reversed)
        # Set ball start position based on meta state
        ball['y'] = self.state[serving_player]['height'] / 2 + self.state[serving_player]['y']
        ball['x'] = self.state[serving_player]['width'] + ball['radius'] if server_is_left else COURT_WIDTH - self.state[serving_player]['width'] - ball['radius']
        ball['speed'] = INITIAL_BALL_SPEED
        ball['serve_mode'] = True
        ball['score_mode'] = False
        ball['score_mode_timeout'] = 0
        self.state['stats']['rally_length'] = 0

    def step(self, action):
        # Encode actions
        button_pressed = action[0] > 0.5  # First dimension is button_pressed
        paddle_direction = action[1]     # Second dimension is paddle_direction
        model_player_actions = {'button_pressed': button_pressed, 'paddle_direction': paddle_direction}
        computer_player_actions = self.get_computer_player_actions(self.state['opponent'])
        actions = {self.state['opponent']:computer_player_actions,self.state['player']:model_player_actions }
        
        # Update state and collect reward
        reward = self.update_game_state(self.state, actions, 0.8)
        obs = self._get_obs()
        info = {}
        terminated = self.check_done()
        truncated = False  # Add logic if you want to support truncation

        return obs, reward, terminated, truncated, info

    def update_game_state(self, actions, delta_time):
        reward = 0
        game_state = self.state
        ball = game_state['ball']
        stats = game_state['stats']
        server = game_state['server']
        paddle_left, paddle_right = (game_state[Player.Player2],game_state[Player.Player1]) if game_state['positions_reversed'] else (game_state[Player.Player1],game_state[Player.Player2])
        model_is_left = (game_state['player'] == Player.Player1 and not game_state['positions_reversed']) or (game_state['player'] == Player.Player2 and game_state['positions_reversed'])
        if ball['score_mode']:
            if ball['score_mode_timeout'] < 50:
                ball['score_mode_timeout'] += delta_time
        elif ball['serve_mode']:
            serving_from_left = (server == Player.Player1 and not game_state['positions_reversed']) or (server == Player.Player2 and game_state['positions_reversed'])
            if serving_from_left:
                ball['x'] = game_state[server]['width'] + ball['radius']
            else:
                ball['x'] = COURT_WIDTH - game_state[server]['width'] - ball['radius']
            if actions[server]['button_pressed']:
                ball['speed'] = INITIAL_BALL_SPEED
                ball['dx'] = -INITIAL_BALL_SPEED
                ball['serve_mode'] = False
                stats['rally_length'] += 1
                stats['serveSpeed'] = abs(ball['dy']) + abs(ball['dx'])
                stats['server'] = server

                if game_state['player'] == server:
                  reward += rewards_map['serve'](abs(ball['dy']) + abs(ball['dx']))
            ball['dy'] = (game_state[server]['y'] + game_state[server]['height'] / 2 - ball['y']) / PADDLE_SPEED_DIVISOR
            ball['y'] += ball['dy'] * delta_time
        else:
            ball['x'] += ball['dx'] * delta_time
            ball['y'] += ball['dy'] * delta_time

            # Check for collisions with top and bottom walls
            if ball['y'] - ball['radius'] < 0:
                ball['dy'] = -ball['dy']
                ball['y'] = ball['radius']  # Adjust ball position to avoid sticking
            elif ball['y'] + ball['radius'] > COURT_HEIGHT:
                ball['dy'] = -ball['dy']
                ball['y'] = COURT_HEIGHT - ball['radius']  # Adjust ball position to avoid sticking

            # Update ball collision detection and response
            if ball['x'] - ball['radius'] < paddle_left['x'] + paddle_left['width'] and ball['y'] + ball['radius'] > paddle_left['y'] and ball['y'] - ball['radius'] < paddle_left['y'] + paddle_left['height']:
                bounce_angle = get_bounce_angle(paddle_left['y'], paddle_left['height'], ball['y'])
                ball['dx'] = (ball['speed'] + abs(paddle_left['dy']) / PADDLE_CONTACT_SPEED_BOOST_DIVISOR) * math.cos(bounce_angle)
                ball['dy'] = (ball['speed'] + abs(paddle_left['dy']) / PADDLE_CONTACT_SPEED_BOOST_DIVISOR) * -math.sin(bounce_angle)
                ball['x'] = paddle_left['x'] + paddle_left['width'] + ball['radius']  # Adjust ball position to avoid sticking
                ball['speed'] += SPEED_INCREMENT
                stats['rally_length'] += 1
                if paddle_left == game_state['player']:
                  reward += rewards_map["hit_paddle"](stats['rally_length'])
            elif ball['x'] + ball['radius'] > paddle_right['x'] and ball['y'] + ball['radius'] > paddle_right['y'] and ball['y'] - ball['radius'] < paddle_right['y'] + paddle_right['height']:
                bounce_angle = get_bounce_angle(paddle_right['y'], paddle_right['height'], ball['y'])
                ball['dx'] = -(ball['speed'] + abs(paddle_right['dy']) / PADDLE_CONTACT_SPEED_BOOST_DIVISOR) * math.cos(bounce_angle)
                ball['dy'] = (ball['speed'] + abs(paddle_right['dy']) / PADDLE_CONTACT_SPEED_BOOST_DIVISOR) * -math.sin(bounce_angle)
                ball['x'] = paddle_right['x'] - ball['radius']  # Adjust ball position to avoid sticking
                ball['speed'] += SPEED_INCREMENT
                stats['rally_length'] += 1
                if paddle_right == game_state['player']:
                  reward += rewards_map["hit_paddle"](stats['rally_length'])
            # Check for scoring
            if ball['x'] - ball['radius'] < 0:
                ball['scoreMode'] = True
                self.is_done = True
                if model_is_left:
                  reward += rewards_map['conceed_point'](ball, paddle_left, stats['rally_length'])
                else:
                  reward += rewards_map['score_point'](stats['rally_length'])
            elif ball['x'] + ball['radius'] > COURT_WIDTH:
                self.is_done = True
                ball['scoreMode'] = True
                if not model_is_left:
                  reward += rewards_map['conceed_point'](ball, paddle_right, stats['rally_length'])
                else:
                  reward += rewards_map['score_point'](stats['rally_length'])

        if game_state['positions_reversed']:
            game_state[Player.Player1]['dy'] = actions[Player.Player1]['paddle_direction']
            game_state[Player.Player2]['dy'] = -actions[Player.Player2]['paddle_direction']
        else:
            game_state[Player.Player1]['dy'] = -actions[Player.Player1]['paddle_direction']
            game_state[Player.Player2]['dy'] = actions[Player.Player2]['paddle_direction']


        
        game_state[Player.Player1]['y'] += game_state[Player.Player1]['dy'] * delta_time
        game_state[Player.Player2]['y'] += game_state[Player.Player2]['dy'] * delta_time

        if model_is_left:
          reward += rewards_map['paddle_movement'](abs(paddle_left['dy']))
          reward += rewards_map['ball_distance'](ball, paddle_left)
        else:
          reward += rewards_map['ball_distance'](ball, paddle_right)
          reward += rewards_map['paddle_movement'](abs(paddle_right['dy']))


        # Ensure paddles stay within screen bounds
        if paddle_left['y'] < 0:
            paddle_left['y'] = 0
        if paddle_left['y'] + paddle_left['height'] > COURT_HEIGHT:
            paddle_left['y'] = COURT_HEIGHT - paddle_left['height']

        if paddle_right['y'] < 0:
            paddle_right['y'] = 0
        if paddle_right['y'] + paddle_right['height'] > COURT_HEIGHT:
            paddle_right['y'] = COURT_HEIGHT - paddle_right['height']


        reward += 0.01 * stats['rally_length']
        return reward

    def get_computer_player_actions(self, player):
        state = self.state
        is_left = (player == Player.Player1 and not state['positions_reversed']) or (player == Player.Player2 and state['positions_reversed'])
        if state['ball']['scoreMode']:
            return {'button_pressed': False, 'paddle_direction': 0}

        paddle = state[player]
        if state['ball']['serve_mode']:
            if paddle['y'] <= 0 or paddle['y'] + paddle['height'] >= COURT_HEIGHT:
                self.direction = -self.direction
            if self.serve_delay_counter > self.serve_delay:
                return {'button_pressed': True, 'paddle_direction': self.direction}
            else:
                self.serve_delay_counter += 1
                return {'button_pressed': False, 'paddle_direction': self.direction}


        if is_left:
          return {
              'button_pressed': False,
              'paddle_direction': self.bounded_value(
                  paddle['y'] - state['ball']['y'] + paddle['height'] / 2 + (np.random.rand() - 0.5) * 2,
                  -MAX_COMPUTER_PADDLE_SPEED,
                  MAX_COMPUTER_PADDLE_SPEED
              )
          }
        else:
          return {
              'button_pressed': False,
              'paddle_direction': -self.bounded_value(
                  paddle['y'] - state['ball']['y'] + paddle['height'] / 2 + (np.random.rand() - 0.5) * 2,
                  -MAX_COMPUTER_PADDLE_SPEED,
                  MAX_COMPUTER_PADDLE_SPEED
              )
          }

    def bounded_value(self, value, min_value, max_value):
        return max(min_value, min(max_value, value))

    def _get_obs(self):
        state = self.state
        player = state['player']
        is_server = 1 if self.state['server'] == player else 0
        paddle = state[player]
        return np.array([
            float(state['ball']['x']),
            float(state['ball']['y']),
            float(state['ball']['dx']),
            float(state['ball']['dy']),
            float(paddle['x']),
            float(paddle['y']),
            float(int(state['ball']['serve_mode'])),
            float(is_server),
        ], dtype=np.float32)


    def check_done(self):
        # Determine if the episode is done
        if self.state['stats']['rally_length'] > 100:
          return True
        return self.is_done

    def render(self, mode='human', close=False):
        if close:
            plt.close()
            return

        if not hasattr(self, 'fig'):
            self.fig, self.ax = plt.subplots()
            self.ax.set_xlim(0, COURT_WIDTH)
            self.ax.set_ylim(0, COURT_HEIGHT)
            self.ax.set_aspect('equal')
            plt.gca().invert_yaxis()  # Invert y-axis to match the coordinate system

        self.ax.clear()
        self.ax.set_xlim(0, COURT_WIDTH)
        self.ax.set_ylim(0, COURT_HEIGHT)

        # Draw paddles
        paddle1 = self.state[Player.Player1]
        paddle2 = self.state[Player.Player2]


        self.ax.add_patch(patches.Rectangle((paddle1['x'], paddle1['y']), paddle1['width'], paddle1['height'], color=paddle1['colour']))
        self.ax.add_patch(patches.Rectangle((paddle2['x'], paddle2['y']), paddle2['width'], paddle2['height'], color=paddle2['colour']))

        # Draw ball
        ball = self.state['ball']
        self.ax.add_patch(patches.Circle((ball['x'], ball['y']), ball['radius'], color='black'))

        # Capture the frame
        plt.draw()
        frame_path = f'frames/frame_{self.frame_count:04d}.png'
        self.fig.savefig(frame_path)
        self.frame_count += 1

    def close(self):
        if hasattr(self, 'fig'):
            plt.close(self.fig)

        # Check if frames directory exists
        if not os.path.exists('frames'):
            print("No frames directory found, skipping video creation.")
            return

        # Create GIF from frames
        with imageio.get_writer('pong_game.gif', mode='I', duration=0.005) as writer:
            for filename in sorted(glob.glob('frames/frame_*.png')):
                image = imageio.imread(filename)
                writer.append_data(image)

        # Display the GIF
        display(Image(filename='pong_game.gif'))

        # Remove frames
        for file in os.listdir('frames'):
            os.remove(os.path.join('frames', file))
        os.rmdir('frames')




: 

In [ ]:
# Test the custom environment
env = CustomPongEnv()
obs = env.reset()
print("Initial observation:", obs)

for i in range(100):
    action = env.action_space.sample()  # Sample random action
    obs, reward, done, info, _ = env.step(action)
    print("Action taken:", action)
    print("Observation:", obs)
    print("Reward:", reward)
    print('iteration:', i)
    print("Done:", done)
    env.render()
    if done:
        break

env.close()

: 

In [ ]:
# Initialize the custom environment with the updated reward function
env = CustomPongEnv()

# Check the environment (optional, to ensure the environment follows Gym's API)
check_env(env, warn=True)
iteration = 0
name = "e"
path = "/content/drive/MyDrive/wimblepong"
# Load the previously trained model or initialize a new one
try:
    model = PPO.load(f"{path}/ppo_custom_pong_{name}.{iteration}", env=env)
    # model = PPO.load(f"./logs/best_model", env=env)
except:
    print('creating new model')
    model = PPO("MlpPolicy", env, verbose=1, learning_rate=3e-4, n_steps=2048, batch_size=64, gamma=0.99, ent_coef=0.01)

# Define evaluation callback
eval_callback = EvalCallback(env, best_model_save_path='./logs/',
                             log_path='./logs/', eval_freq=1000,
                             deterministic=False, render=False)

# Continue training the model with more iterations
model.learn(total_timesteps=100000, callback=eval_callback)

# Save the retrained model
model.save(f"{path}/ppo_custom_pong_{name}.{iteration + 1}")

# Evaluate the retrained model
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward} +/- {std_reward}")


: 

In [ ]:
# Evaluate the trained model
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward} +/- {std_reward}")

: 

In [ ]:
env = CustomPongEnv()
# model = PPO.load(f"{path}/ppo_custom_pong_{name}.{iteration+1}")
model = PPO.load(f"./logs/best_model")

obs, _ = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs)
    obs, rewards, done, info, _ = env.step(action)
    print(rewards)
    env.render()
    if done:
        break
        print("resetting due to done")
        print(env.state)
        obs, _ = env.reset()

env.close()


: 

In [ ]:
# Load the trained PPO model
model = PPO.load(f"{path}/ppo_custom_pong_{iteration+1}")

# Extract the PyTorch model's weights
policy_weights = model.policy.state_dict()

# Define the TensorFlow model
class TfPPOPolicy(tf.Module):
    def __init__(self, input_dim, output_dim):
        super(TfPPOPolicy, self).__init__()
        self.fc1 = layers.Dense(64, activation='tanh', input_shape=(input_dim,))
        self.fc2 = layers.Dense(64, activation='tanh')
        self.action_head = layers.Dense(output_dim)
        self.value_head = layers.Dense(1)

    @tf.function(input_signature=[tf.TensorSpec(shape=[None, 9], dtype=tf.float32)])
    def __call__(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        action_logits = self.action_head(x)
        value = self.value_head(x)
        return action_logits, value

    def predict(self, x):
        action_logits, value = self.__call__(x)
        # Apply sigmoid to the buttonPressed logits to constrain between 0 and 1
        buttonPressed = tf.sigmoid(action_logits[:, 0])
        paddleDirection = action_logits[:, 1]
        return buttonPressed, paddleDirection


# Create the TensorFlow model
input_dim = 9  # Your observation space dimension
output_dim = 2  # Number of actions
tf_policy = TfPPOPolicy(input_dim, output_dim)

# Load weights into the TensorFlow model
tf_policy.fc1.build((None, input_dim))
tf_policy.fc1.set_weights([
    policy_weights['mlp_extractor.policy_net.0.weight'].cpu().numpy().T,
    policy_weights['mlp_extractor.policy_net.0.bias'].cpu().numpy()
])

tf_policy.fc2.build((None, 64))
tf_policy.fc2.set_weights([
    policy_weights['mlp_extractor.policy_net.2.weight'].cpu().numpy().T,
    policy_weights['mlp_extractor.policy_net.2.bias'].cpu().numpy()
])

tf_policy.action_head.build((None, 64))
tf_policy.action_head.set_weights([
    policy_weights['action_net.weight'].cpu().numpy().T,
    policy_weights['action_net.bias'].cpu().numpy()
])

tf_policy.value_head.build((None, 64))
tf_policy.value_head.set_weights([
    policy_weights['value_net.weight'].cpu().numpy().T,
    policy_weights['value_net.bias'].cpu().numpy()
])

# Save the wrapped policy as a TensorFlow SavedModel
saved_model_path = "./saved_model"
tf.saved_model.save(tf_policy, saved_model_path)

# Convert the SavedModel to TensorFlow.js format using the command line
# !tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model ./saved_model ./tfjs_model


: 

In [ ]:
# Convert the SavedModel to TensorFlow.js format using the command line
!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model ./saved_model ./tfjs_model

: 